# Gender Classification CNN (VGG16, VGG-Face Transfer Learning)

### Import Dependencies

In [1]:
import os
import numpy as np
from numpy import array
from numpy import argmax
import imageio
import matplotlib.pyplot as plt
import pandas as pd
import glob
import keras
import time
from keras import backend as K
from keras.layers.core import Dense
from keras.layers import Convolution2D, MaxPooling2D, Dropout
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
from keras.optimizers import SGD
from keras.callbacks import CSVLogger, EarlyStopping, TensorBoard, ModelCheckpoint 
import matplotlib.pyplot as plt
from scipy.misc import imread, imresize
from keras.applications.vgg16 import VGG16
from keras_vggface.vggface import VGGFace
from keras.preprocessing import image
from keras.layers import Input, Flatten, Dense
from keras.models import Model
from keras import backend as K
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
%matplotlib inline



Using TensorFlow backend.


## Preprocessing (Data Augmentation Approach)

In [2]:
#Import labels
image_labels = pd.read_csv('train_target.csv')

In [5]:
#prepare data for flow_from_dir
from shutil import copyfile

#Split training and testing 80% and 20% respectively
data_size = len(image_labels.index)
train_size = data_size - (data_size * 0.2)
count = 0

#Iterate through labels to create directories (data/train) and (data/validation) separated by classes
for index, row in image_labels.iterrows():
    name = row['Id']
    gender = str(row['Gender']) 

    #For the first 80%, copy to data/train
    if count<train_size:
        
        #Check if dir exists then copy the image to the directory, otherwise create the directory and copy the image over.
        if os.path.isdir('data/train/' + gender):
            copyfile('train/'+name, 'data/train/'+gender+'/'+name)
        else:
            os.makedirs('data/train/'+gender)
            copyfile('train/'+name, 'data/train/'+gender+'/'+name)

        count = count + 1

    #For the last 20% of data, copy to validation with same logic as previous.
    else:
        if os.path.isdir('data/validation/' + gender):
            copyfile('train/'+name, 'data/validation/'+gender+'/'+name)
        else:
            os.makedirs('data/validation/'+gender)
            copyfile('train/'+name, 'data/validation/'+gender+'/'+name)
    

In [ ]:
#Helper Function to plot the Image
def plotImage(image):
    f, axarr = plt.subplots(1,2)
    axarr[0].imshow(image)
    axarr[0].grid()
    axarr[0].set_title('Image')

## Preprocessing (Standard Fitting Approach/No augmentation)

In [19]:
#Create collection for dataset, read the image using names from csv file and append.
X_train = []
Y_train = []
for index, row in image_labels.iterrows():
    image = imread('train/' + row[0], mode='RGB')
    image = imresize(image, (224,224,3))
    X_train.append(np.array(image))
    Y_train.append(row[1])

C:\Users\user\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:5: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
  """
C:\Users\user\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.
  


In [20]:
#convert to np array
X_train = np.array(X_train)

In [21]:
#convert to one-hot representation
Y_train = to_categorical(Y_train)

In [22]:
#Split data into training and testing
X_train, X_test, y_train, y_test = train_test_split(X_train, Y_train, test_size=0.25, random_state=3)

In [23]:
#Number of classes
num_classes = np.size(Y_train, 1)

## VGG-Face Model (#1)

In [7]:
#Create the VGG Face model
vgg_face_model = VGGFace(model = 'resnet50', include_top = False, weights='vggface', input_shape=((224,224,3)))
    
vgg_face_model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1/7x7_s2 (Conv2D)           (None, 112, 112, 64) 9408        input_1[0][0]                    
__________________________________________________________________________________________________
conv1/7x7_s2/bn (BatchNormaliza (None, 112, 112, 64) 256         conv1/7x7_s2[0][0]               
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 112, 112, 64) 0           conv1/7x7_s2/bn[0][0]            
__________________________________________________________________________________________________
max_poolin

In [8]:
#Freeze all layers
for layer in vgg_face_model.layers[:-1]:
    layer.trainable = False
    

In [9]:
#Add the fully connected layers and classifier
LL = vgg_face_model.get_layer('avg_pool').output
x = Flatten(name='flatten')(LL)
x = Dense(4096,name = 'fc6')(x)
x = Dropout(0.5)(x)
x = Dense(4096,name = 'fc7')(x)
x = Dropout(0.5)(x)
x = Dense(2622,name = 'fc8')(x)
out = Dense(1, activation='sigmoid',name='classifier')(x)
custom_vgg_face_model = Model(vgg_face_model.input, out)

In [10]:
custom_vgg_face_model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1/7x7_s2 (Conv2D)           (None, 112, 112, 64) 9408        input_1[0][0]                    
__________________________________________________________________________________________________
conv1/7x7_s2/bn (BatchNormaliza (None, 112, 112, 64) 256         conv1/7x7_s2[0][0]               
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 112, 112, 64) 0           conv1/7x7_s2/bn[0][0]            
__________________________________________________________________________________________________
max_poolin

In [11]:
#Specify optimizer, loss function, and metrics to track
sgd = SGD(lr=0.001, decay=5e-4, momentum=0.9)
custom_vgg_face_model.compile(optimizer=sgd, loss='binary_crossentropy', metrics=['accuracy'])

In [3]:
from keras.preprocessing.image import ImageDataGenerator
batch_size = 64

#Data Augmentation, feed images to the model with modifications.
train_datagen = ImageDataGenerator(
        rotation_range=20,
        width_shift_range=0.1,
        height_shift_range=0.1,
        shear_range=0.1,
        zoom_range=0.1,
        horizontal_flip=True,
        vertical_flip=False)

test_datagen = ImageDataGenerator(
        rotation_range=20,
        width_shift_range=0.1,
        height_shift_range=0.1,
        shear_range=0.1,
        zoom_range=0.1,
        horizontal_flip=True,
        vertical_flip=False)


train_generator = train_datagen.flow_from_directory(
        'data/train', 
        target_size=(224,224),  
        batch_size=batch_size,
        color_mode="rgb",
        class_mode='binary')  

validation_generator = test_datagen.flow_from_directory(
        'data/validation', 
        target_size=(224,224),  
        batch_size=batch_size,
        color_mode="rgb",
        class_mode='binary')


Found 22681 images belonging to 2 classes.
Found 5672 images belonging to 2 classes.


In [77]:
# fine-tune the model
filepath="vgg_face_gender_weights_improvment-{epoch:02d}-{val_acc:.2f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
data_size = len(image_labels.index)
train_size = data_size - (data_size * 0.2)
#tensorboard = TensorBoard(log_dir=".", histogram_freq=2000, write_graph=True, write_images=False)
callback_list = [checkpoint]

custom_vgg_face_model.fit_generator(
        train_generator,
        steps_per_epoch=train_size // batch_size,
        validation_data=validation_generator,
        validation_steps=(data_size - train_size)  // batch_size, callbacks=callback_list, epochs=75, verbose = 1)

Epoch 1/75
354/354 [==============================] - 320s 903ms/step - loss: 0.6219 - acc: 0.8532 - val_loss: 0.3157 - val_acc: 0.9123

Epoch 00001: val_acc improved from -inf to 0.91229, saving model to vgg_face_gender_weights_improvment-01-0.91.hdf5
Epoch 2/75
354/354 [==============================] - 313s 883ms/step - loss: 0.3454 - acc: 0.8812 - val_loss: 0.3294 - val_acc: 0.9119

Epoch 00002: val_acc did not improve from 0.91229
Epoch 3/75
354/354 [==============================] - 356s 1s/step - loss: 0.3110 - acc: 0.8922 - val_loss: 0.3195 - val_acc: 0.9107

Epoch 00003: val_acc did not improve from 0.91229
Epoch 4/75
354/354 [==============================] - 390s 1s/step - loss: 0.2904 - acc: 0.8998 - val_loss: 0.3178 - val_acc: 0.9132

Epoch 00004: val_acc improved from 0.91229 to 0.91317, saving model to vgg_face_gender_weights_improvment-04-0.91.hdf5
Epoch 5/75
354/354 [==============================] - 311s 878ms/step - loss: 0.2799 - acc: 0.9016 - val_loss: 0.2823 - val

KeyboardInterrupt: 

# VGG-FACE (Experiment 2)
## Attempting regularization

In [23]:
#Create the VGG Face model
vgg_face_model = VGGFace(model = 'resnet50', include_top = False, weights='vggface', input_shape=((224,224,3)))
    
vgg_face_model.summary()

#Freeze all layers
for layer in vgg_face_model.layers[:-1]:
    layer.trainable = False

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1/7x7_s2 (Conv2D)           (None, 112, 112, 64) 9408        input_3[0][0]                    
__________________________________________________________________________________________________
conv1/7x7_s2/bn (BatchNormaliza (None, 112, 112, 64) 256         conv1/7x7_s2[0][0]               
__________________________________________________________________________________________________
activation_99 (Activation)      (None, 112, 112, 64) 0           conv1/7x7_s2/bn[0][0]            
__________________________________________________________________________________________________
max_poolin

In [24]:
#Add the fully connected layers and classifier
from keras import regularizers
LL = vgg_face_model.get_layer('avg_pool').output
x = Flatten(name='flatten')(LL)
x = Dense(4096, kernel_regularizer=regularizers.l2(0.0003), activity_regularizer=regularizers.l1(0.0003), name = 'fc5')(x)
x = Dropout(0.5)(x)
x = Dense(4096, kernel_regularizer=regularizers.l2(0.0004), activity_regularizer=regularizers.l1(0.0004), name = 'fc6')(x)
x = Dropout(0.5)(x)
x = Dense(2622,name = 'fc7')(x)
out = Dense(1, activation='sigmoid',name='classifier')(x)
custom_vgg_face_model = Model(vgg_face_model.input, out)

In [25]:
#Specify optimizer, loss function, and metrics to track
sgd = SGD(lr=0.0002, decay=5e-4, momentum=0.9)
custom_vgg_face_model.compile(optimizer=sgd, loss='binary_crossentropy', metrics=['accuracy'])

In [27]:
# fine-tune the model
filepath="vgg_face_gender_weights_improvment-{epoch:02d}-{val_acc:.2f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
data_size = len(image_labels.index)
train_size = data_size - (data_size * 0.2)
#tensorboard = TensorBoard(log_dir=".", histogram_freq=2000, write_graph=True, write_images=False)
callback_list = [checkpoint]

custom_vgg_face_model.fit_generator(
        train_generator,
        steps_per_epoch=train_size // batch_size,
        validation_data=validation_generator,
        validation_steps=(data_size - train_size)  // batch_size, callbacks=callback_list, epochs=30, verbose = 1)

Epoch 1/30
354/354 [==============================] - 317s 896ms/step - loss: 168.3211 - acc: 0.8096 - val_loss: 126.0694 - val_acc: 0.9162

Epoch 00001: val_acc improved from -inf to 0.91619, saving model to vgg_face_gender_weights_improvment-01-0.92.hdf5
Epoch 2/30
354/354 [==============================] - 311s 878ms/step - loss: 126.3547 - acc: 0.8438 - val_loss: 106.8401 - val_acc: 0.9148

Epoch 00002: val_acc did not improve from 0.91619
Epoch 3/30
354/354 [==============================] - 310s 874ms/step - loss: 108.8583 - acc: 0.8573 - val_loss: 95.7134 - val_acc: 0.9041

Epoch 00003: val_acc did not improve from 0.91619
Epoch 4/30
 25/354 [=>............................] - ETA: 2:27 - loss: 102.5540 - acc: 0.8744

KeyboardInterrupt: 

# Prediction

## Preload an already existing model JSON

In [64]:
from keras.models import load_model
from keras.models import model_from_json
json_file = open('vgg_face_trained.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)

In [16]:
#load weights if needed
custom_vgg_face_model.load_weights("vgg_face_first_place-0.92.hdf5")
print("Loaded model from disk")

Loaded model from disk


In [57]:
#read file names from sample_submissions
test_labels = pd.read_csv('sample_submission.csv')

In [58]:
#append the images to a collection after reading files and converting to np arrays
X_testing_data = []
X_names = []
for index, row in test_labels.iterrows():
    X_names.append(row[0])
    image = imread('test/' + row[0], mode='RGB')
    image = imresize(image, (224,224,3))
    X_testing_data.append(np.array(image))


C:\Users\user\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
  
C:\Users\user\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:7: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.
  import sys


In [59]:
#convert to np array
testing_data = np.asarray(X_testing_data)

In [60]:
#Make predictions
predictions = custom_vgg_face_model.predict(testing_data)

In [35]:
#evaluate accuracy of model
custom_vgg_face_model.evaluate(X_test,np.argmax(y_test, axis=1))

7090/7090 [==============================] - 37s 5ms/step


[0.25158107737223756, 0.9280677010041198]

In [61]:
#Convert to a single dataframe and prepare for export
names = pd.DataFrame(X_names)  
gender = pd.DataFrame(predictions) 
result = pd.concat([names, gender],axis=1)
result.columns = ['Id', 'Expected']

In [62]:
#Save results to CSV
result.to_csv('answers_op2_vgg_face_testing_5.csv')